In [3]:
import numpy as np 
import cv2
import time
import imageio, imageio_ffmpeg
from IPython.display import display, clear_output

In [11]:
def draw_flow(img, flow, step=8, threshold=1.0):  # Added threshold parameter
    scale = 5
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    magnitude = np.sqrt(fx**2 + fy**2)
    mask = magnitude > threshold  # Filter out vectors below the threshold
    fx, fy = fx[mask] * scale, fy[mask] * scale  # Scale the vectors
    lines = np.vstack([x[mask], y[mask], x[mask]-fx, y[mask]-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)
    return img_bgr

def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr

In [10]:
cap = cv2.VideoCapture(0)
suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

while True:
    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    start = time.time()
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 5, 15, 3, 7, 1.5, 0)  # Adjust parameters for sensitivity
    prevgray = gray
    end = time.time()
    fps = 1 / (end-start)
    clear_output(wait=True)
    print(f"{fps:.2f} FPS")
    cv2.imshow('flow', draw_flow(gray, flow, step=8, threshold=1.0))  # Pass threshold to draw_flow
    cv2.imshow('flow HSV', draw_hsv(flow))
    key = cv2.waitKey(5)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

14.64 FPS


In [27]:
# Read the MP4 video
video_path = 'LavaLamp.mp4'
video = imageio.get_reader(video_path, 'ffmpeg')

# Convert frames to grayscale
gray_frames = [cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) for frame in video]

# Initialize the previous frame
prevgray = gray_frames[0]

# Process each frame
processed_frames = []
for gray in gray_frames[1:100]:
    start = time.time()
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 5, 15, 3, 7, 1.5, 0)  # Adjust parameters for sensitivity
    prevgray = gray
    end = time.time()
    fps = 1 / (end-start)
    clear_output(wait=True)
    print(f"{fps:.2f} FPS")
    flow_frame = draw_flow(gray, flow, step=8, threshold=1.0)  # Pass threshold to draw_flow
    processed_frames.append(flow_frame)

# Save the processed frames as a new GIF
output_gif_path = 'output.gif'
imageio.mimsave(output_gif_path, processed_frames, duration=0.1)

1.27 FPS


In [12]:
# Read the GIF
gif_path = 'Plop.gif'
gif = imageio.mimread(gif_path, memtest=False)

# Convert frames to grayscale
gray_frames = [cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) for frame in gif]

# Initialize the previous frame
prevgray = gray_frames[0]

# Process each frame
processed_frames = []
for gray in gray_frames[1:]:
    start = time.time()
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 5, 15, 3, 7, 1.5, 0)  # Adjust parameters for sensitivity
    prevgray = gray
    end = time.time()
    fps = 1 / (end-start)
    clear_output(wait=True)
    print(f"{fps:.2f} FPS")
    flow_frame = draw_flow(gray, flow, step=8, threshold=1.0)  # Pass threshold to draw_flow
    processed_frames.append(flow_frame)

# Save the processed frames as a new GIF
output_gif_path = 'OutputPlop.gif'
imageio.mimsave(output_gif_path, processed_frames, duration=0.1)

12.33 FPS


In [2]:
pip list --format=freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
